In [2]:
%%html
<style>
body,h1,h2,h3,h4,h5,p,div{font-family:'ubuntu' !important};
.CodeMirror-line{font-family:'Fira Sans Book' !important}
</style>


<h1>Kubernetes</h1>



<img src="assets/k8s-logo.png" alt="k8s-logo" style="width: 400px;"/>

### Agenda
* Was ist Kubernetes?
* Grundlagen
* kubectl
* Deployment Beispiel

# Was ist Kubernetes


* opensource Programm zum automatisierten Deployment

* basierend auf der Container Runtime von Docker

* cloud-native

# verteilt

* Kubernetes verteilt Deployments über verschiedene Rechner
* eine Instanz heißt Cluster und besteht aus mehreren Nodes (Rechner)

# zuverlässig und hochverfügbar
* Kubernetes sorgt selbst dafür, dass Deployments laufen (self-healing)
* Deployments erfolgen ohne Downtime

# skalierbar
* Deployments können je nach Last automatisch skaliert werden
* Clusterresourcen können dynamisch erweitert werden (Nodes hinzufügen)

# Kubernetes und Microservices
K8s begünstigt Micorservice Architekturen  
-> und somit agile Arbeitsweisen 

# Grundlagen

<h1> Deklarative Konfiguration  <br>oder Infrastructure as code</h1>





* alles in K8s sind API-Resourcen
* Interaktionen mit Kubernetes erfolgen über diese API (kubectl)
* Befehle können ausgeführt werden

* **besser:** keine Anweisungen ausführen, sondern nur Config-Dateien bereitstellen
* Verwaltung der Config-Dateien mit VCS
* Ziel: Zustand des Clusters entspricht dem in der Versionsverwaltung

# kubectl
* cli für die K8s Api

In [6]:
! kubectl version

Client Version: version.Info{Major:"1", Minor:"19", GitVersion:"v1.19.3", GitCommit:"1e11e4a2108024935ecfcb2912226cedeafd99df", GitTreeState:"clean", BuildDate:"2020-10-14T12:50:19Z", GoVersion:"go1.15.2", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"19", GitVersion:"v1.19.2", GitCommit:"f5743093fd1c663cb0cbc89748f730662345d44d", GitTreeState:"clean", BuildDate:"2020-09-16T13:32:58Z", GoVersion:"go1.15", Compiler:"gc", Platform:"linux/amd64"}


# lokaler K8s Cluster mit Minikube

In [5]:
! minikube status

minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



# ein paar Kubernetes Resourcen


<img src="assets/ressources.png"  style="max-height:90vh"/>

# Pod
* enthält einen oder mehrer Container
* Conatainer teilen sich Netzwerk und eventuell Speicher
* kleinste deploybare Einheit

In [6]:
! kubectl get pods

NAME                                           READY   STATUS    RESTARTS   AGE
filefighter-db-deployment-7f64f99568-46svj     1/1     Running   0          45m
filefighter-fe-deployment-65b58f495f-56dhp     1/1     Running   1          44h
filefighter-fe-deployment-65b58f495f-vgnk7     1/1     Running   1          44h
filefighter-rest-deployment-775cf944d7-zfdgs   1/1     Running   4          48m


In [4]:
! kubectl describe pods -l app=filefighter-db

Name:           filefighter-db-deployment-7f64f99568-46svj
Namespace:      default
Priority:       0
Node:           minikube/192.168.49.2
Start Time:     Thu, 05 Nov 2020 10:43:08 +0100
Labels:         app=filefighter-db
                pod-template-hash=7f64f99568
Annotations:    <none>
Status:         Running
IP:             
IPs:            <none>
Controlled By:  ReplicaSet/filefighter-db-deployment-7f64f99568
Containers:
  ff-db:
    Container ID:   docker://300b2fb122dbf6502e5741b5f19864c5abd26a30074e89b76442e6c3bbe57e6a
    Image:          mongo:latest
    Image ID:       docker-pullable://mongo@sha256:efc408845bc917d0b7fd97a8590e9c8d3c314f58cee651bd3030c9cf2ce9032d
    Port:           27017/TCP
    Host Port:      0/TCP
    State:          Terminated
      Reason:       Completed
      Exit Code:    0
      Started:      Thu, 05 Nov 2020 10:43:10 +0100
      Finished:     Thu, 05 Nov 2020 11:51:18 +0100
    Ready:          False
    Restart Count:  0
    Environment:
      MONG

# Deployment
* gibt den gewünschten Zustand eines Deployments an
* beschreibt Pods
* ermöglicht Sklarierung (ReplicaSets)

In [6]:
! kubectl get deployments

NAME                          READY   UP-TO-DATE   AVAILABLE   AGE
filefighter-db-deployment     1/1     1            1           6h40m
filefighter-fe-deployment     2/2     2            2           2d3h
filefighter-rest-deployment   1/1     1            1           2d3h


In [7]:
! kubectl describe deployment filefighter-db-deployment

Name:                   filefighter-db-deployment
Namespace:              default
CreationTimestamp:      Thu, 05 Nov 2020 10:43:08 +0100
Labels:                 app=filefighter
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=filefighter-db
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=filefighter-db
  Containers:
   ff-db:
    Image:      mongo:latest
    Port:       27017/TCP
    Host Port:  0/TCP
    Environment:
      MONGO_INITDB:                filefighter
      MONGO_INITDB_ROOT_USERNAME:  root
      MONGO_INITDB_ROOT_PASSWORD:  <set to the key 'password' in secret 'ff-secret'>  Optional: false
    Mounts:                        <none>
  Volumes:                         <none>
Conditions:
  Type           Status  Reason
  ----           ------  -----

# Services
* Addressierung der Pods
* inklusive Loadbalancing
* ermöglicht z.B. Kommunikation zwischen Pods

<img src="assets/services.png"  style="max-height:60vh"/>

In [9]:
! kubectl get services

NAME              TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
ff-db-service     ClusterIP      10.97.167.246   <none>          27017/TCP      2d3h
ff-fe-service     LoadBalancer   10.105.66.133   10.105.66.133   80:32400/TCP   2d23h
ff-rest-service   ClusterIP      10.96.82.25     <none>          80/TCP         2d23h
kubernetes        ClusterIP      10.96.0.1       <none>          443/TCP        3d5h


# Volumes
* da Container zustandlos sind müssen die Daten an anderer Stelle gespeichert werden
* können von Containern gemountet werden
* gibt verschiedene Volumetypen je nach Anwendungsfall


# Secrets
* zum Speichern senitiver Daten
* werden in Pods eingebunden
* entweder als Datei oder als Umgebungsvariablen

In [3]:
! kubectl get secrets

The connection to the server 192.168.49.2:8443 was refused - did you specify the right host or port?


# Deployment Beispiel

# FileFighter

<img src="assets/ff-logo.png" alt="k8s-logo" style="max-width: 400px;"/>

# Architektur
<img src="assets/arch2.svg" alt="k8s-logo" style="width:1000px"/>